# Google Cloud Platform - Using Machine Learning APIs  ).

This is an upgraded Python revision of [this notebook](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/CPB100/lab4c/mlapis.ipynb).

This notebook originally was being processed using DataLab on the Google Cloud Platform.  This particular incarnation of the notebook is for running on Google Colaboratory which I am trying out for the first time.

### Security

First things first - we need to authenticate against the Google Cloud APIs.

#### Getting a Google API Credential.

First, visit <a href="http://console.cloud.google.com/apis">API console</a>, choose "Credentials" on the left-hand menu.  Choose "Create Credentials" and generate an API key for your application. You should probably restrict it by IP address to prevent abuse, but for now, just  leave that field blank and delete the API key after trying out this demo.

Then, when you have your key, you will enter it in this first executable cell:

In [0]:

'''
!pip install -q cloudstorage
!pip install -q webapp2
!pip install --upgrade numpy
!pip install six==1.10.0
!pip install -q datalab
!pip install --upgrade gcs-client
!pip install --upgrade google-cloud-storage
!pip install --upgrade google-api-python-client
!pip install opencv-python
!pip install gcsfs
'''

'\n!pip install -q cloudstorage\n!pip install -q webapp2\n!pip install --upgrade numpy\n!pip install six==1.10.0\n!pip install -q datalab\n!pip install --upgrade gcs-client\n!pip install --upgrade google-cloud-storage\n!pip install --upgrade google-api-python-client\n!pip install opencv-python\n!pip install gcsfs\n'

In [2]:
import getpass

APIKEY = getpass.getpass()

··········


In [3]:
import sys
import json
import argparse
import googleapiclient.discovery
import pandas as pd
import skimage.data
from skimage import img_as_float
import numpy as np
import pandas as pd
import os
import sys
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
%matplotlib inline
import io


import sys
import json
import argparse
import googleapiclient.discovery
import pandas as pd
import skimage.data
from skimage import img_as_float
import numpy as np
import pandas as pd
import os
import sys
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
%matplotlib inline
import io
from google.cloud import storage


import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
import matplotlib.image as mpimg


Using TensorFlow backend.


In [0]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

# Create the service client.
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')




In [0]:
BUCKET='wpiopenimageskaggle'


In [0]:
def create_service():
    return googleapiclient.discovery.build('storage', 'v1')

In [0]:
def download_content(filename,BUCKET):
    from google.cloud import storage
    client = storage.Client('MLRobustClassifier')
    bucket = client.get_bucket(BUCKET)
    blob = storage.Blob(filename, bucket)
    content = blob.download_as_string()
    return(content)

In [0]:
def plot_image(filename):
  from google.cloud import storage
  client = storage.Client('MLRobustClassifier')
  bucket = client.get_bucket(BUCKET)
  blob = storage.Blob(filename, bucket)
  content = blob.download_as_string()
  img = Image.open(io.BytesIO(content))
  import matplotlib.pyplot as plt

  plt.imshow(img)
  plt.show()

In [0]:
def strip_filepath(infiles):
    strip=[]
    for path in infiles:
        res=path.split("/")
        res=res[1]
        res=res.split('.')
        res=res[0]
        strip.append(res)
    files=pd.DataFrame()
    files['ImageID']=strip
    files['Url']=infiles
    return(files)

In [0]:
def list_bucket_pf(bucket, pf):
    """Returns a list of metadata of the objects within the given bucket."""
    service = create_service()

    # Create a request to objects.list to retrieve a list of objects.
    fields_to_return = 'nextPageToken,items(name,size,contentType,metadata(my-key))'
    req = service.objects().list(bucket=bucket, fields=fields_to_return, prefix=pf)  # returns everything. UrbanSound is top dir in bucket

    all_objects = []
    # If you have too many items to list in one request, list_next() will
    # automatically handle paging with the pageToken.
    while req:
        resp = req.execute()
        all_objects.extend(resp.get('items', []))
        req = service.objects().list_next(req, resp)
    return all_objects



In [0]:
def process_image(filename):
  from google.cloud import storage
  client = storage.Client('MLRobustClassifier')
  bucket = client.get_bucket(BUCKET)
  blob = storage.Blob(filename, bucket)
  content = blob.download_as_string()
  img = Image.open(io.BytesIO(content)).convert('L')
  img=np.array(img)
  return(img)

In [0]:
def CreateDictLabels(Points):
    dictOfLabels = {}
    for point in Points:
        labelkey = point['annotations'][0]['label'] #get the label
        if labelkey not in dictOfLabels:
            dictOfLabels.setdefault(labelkey, [])
        dictOfLabels[labelkey].append(point['id'])
    return dictOfLabels

In [0]:
def CreateBatchOfImages(batchSize, labelDict):
    global imageInputPath
    ImageLabelDict = {}
    SuccessNum = 0
    for index in range(batchSize):
        for labelkey in labelDict.keys():
            if labelkey not in ImageLabelDict:
                ImageLabelDict.setdefault(labelkey, [])

            #force add one image, keep trying until one has been added, protects against corrupted images
            while len(labelDict[labelkey]) != 0:
                try:
                    imageInputPath='Imagefiles256x256/'
                    imageId = labelDict[labelkey].pop(0)
                    uriInp = imageInputPath + imageId + ".jpg"
                    print(uriInp)
                    '''
                    with _open_file_read_binary(uriInp) as f:
                        image_bytes = f.read()
                        img = Image.open(io.BytesIO(image_bytes))
                    '''

                    #img= imageio.imread(uriInp)
                        #img = Image.open(io.BytesIO(image_bytes))
                    imgArr =  process_image(uriInp)
                    print(imgArr)
                except: #find image exception error for better style
                    continue
                else:
                    ImageLabelDict[labelkey].append([imgArr, imageId])
                    break
        #make sure all have nonzero number of images left
 
        SuccessNum += 1
    return ImageLabelDict, labelDict, SuccessNum

In [14]:
from google.colab import files
uploaded = files.upload()
file='PointAnnotationsSet256x256.txt'
points = json.loads(uploaded[file].decode('utf-8'))

Saving PointAnnotationsSet256x256.txt to PointAnnotationsSet256x256 (3).txt


In [0]:
IdsFromLabels = CreateDictLabels(points)
idsToRemoveFromEachBatch = IdsFromLabels.copy()

In [16]:
ImageLabelDict, idsToRemoveFromEachBatch, SuccessNum = CreateBatchOfImages(10, idsToRemoveFromEachBatch)

Imagefiles256x256/9eb39d618fd92994.jpg
Imagefiles256x256/ad56d643354b9f53.jpg
Imagefiles256x256/a7a1e58de7ce4154.jpg
Imagefiles256x256/a2d0287ab2de8f2f.jpg
Imagefiles256x256/4dc2c29d6202f84c.jpg
Imagefiles256x256/66449e223b80a0a5.jpg
Imagefiles256x256/7892ad7113275c6b.jpg
Imagefiles256x256/5176cca60e245e3a.jpg
Imagefiles256x256/5219cef5b7f9f7b4.jpg
Imagefiles256x256/f81b23b5fafe03cc.jpg
Imagefiles256x256/b458cec0a7092abe.jpg
Imagefiles256x256/332635c359800b3f.jpg
Imagefiles256x256/d24fb56ef73c5b02.jpg
Imagefiles256x256/0e8ca0aca85b0c59.jpg
Imagefiles256x256/70ad34e6faddbfe9.jpg
Imagefiles256x256/67c4763f585892ce.jpg
Imagefiles256x256/db3b44b6711e7e2c.jpg
Imagefiles256x256/202d0c6ccef99ced.jpg
Imagefiles256x256/ce9f38474ed595a4.jpg
Imagefiles256x256/2aad92b4b57a7b3e.jpg
Imagefiles256x256/bb703e867e1369bd.jpg
Imagefiles256x256/e0cc9abb47f886b0.jpg
Imagefiles256x256/807f839db4fcc4d1.jpg
Imagefiles256x256/c85a9b5b8d81268b.jpg
Imagefiles256x256/a38e9efbe565fc73.jpg
Imagefiles256x256/6cbffcd

In [0]:
keys=list(ImageLabelDict.keys())
data=[]
labels=[]
label=0
for val in keys:
  imgs=ImageLabelDict.get(val)
  for img in imgs:
        vec=img[0]
        data.append(vec)
        labels.append(label)
  label+=1
        
    ########
    ##
    ##Split data intro training and testing wtih 0.75 train vs 0.25 test
    ########
        
total=len(data)
train_size=round(total*0.75)
test_size=total-train_size
import random
random.seed(4)
  
train_ind=np.random.choice(len(labels),train_size,replace=False)
test_ind=np.setdiff1d(list(range(0, total)),train_ind)


labels_test=np.array(labels)
data_test=np.array(data)
    
X_train=data_test[np.where(train_ind)]
X_test=data_test[np.where(test_ind)]
Y_train=labels_test[np.where(train_ind)]
Y_test=labels_test[np.where(test_ind)]

In [0]:

img_len=256
img_width=256
# We reshape the input data to have a depth of 1 (grey scale)
if keras.backend.image_data_format() == 'channels_first':
        X_train = X_train.reshape(X_train.shape[0], 1, img_len, img_width)
        X_test = X_test.reshape(X_test.shape[0], 1, img_len, img_width)
        input_shape = (1, img_len, img_width)
else:
        X_train = X_train.reshape(X_train.shape[0], img_len, img_width, 1)
        X_test = X_test.reshape(X_test.shape[0], img_len, img_width, 1)
        input_shape = (img_len, img_width, 1)


In [0]:
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    # Then we normalize it so that the values are between 0 and 1
    X_train /= 255
    X_test /= 255

In [25]:
   # Defining the model
    model = Sequential()
    model.add(Conv2D(32, 3, 3, activation='relu', input_shape=(img_len, img_width, 1)))
    # 32 is the number of convolutional filters to use. Frist 3 is the number of rows in each convolution kernel and second 3 is the number of columns in each kernel.


    model.add(Conv2D(32, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    # MaxPooling2D is a way to reduce the number of parameters in our model by sliding a 2x2 pooling filter across the previous layer and taking the max of the 4 values in the 2x2 filter.

    model.add(Dropout(0.25))
    # Dropout regularizes the model and prevents overfitting

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    # Final layer has the output size of 10 to correspond to the number of classes

    model.summary()
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit model to training data
    model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)

    # Evaluate the model on test data
    score = model.evaluate(X_test, Y_test, verbose=0)
    print("test loss:", score[0])
    print("test accuracy:",  score[1])
    # score[0] gives you the test loss and score[1] gives you the accuracy

    tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1, write_graph=True, write_grads=True, batch_size=32, write_images=True)

    # We can use a call back to look into the internal state of the model during training
    model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, Y_test), callbacks=[tbCallBack])





/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(256, 256,...)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 126, 126, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 508032)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65028224  
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [43]:
!pip install graphviz
!pip install pydot
!pip install pydotplus
!pip install graphviz 

Invalid requirement: '!apt-get'
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/packaging/requirements.py", line 93, in __init__
    req = REQUIREMENT.parseString(requirement_string)
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pyparsing.py", line 1654, in parseString
    raise exc
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pyparsing.py", line 1644, in parseString
    loc, tokens = self._parse( instring, 0 )
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pyparsing.py", line 1402, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pyparsing.py", line 3417, in parseImpl
    loc, exprtokens = e._parse( instring, loc, doActions )
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pyparsing.py", line 1402, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  File "/usr/local/lib/py

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import argparse
import os
import pydot
import graphviz


In [0]:
# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


def plot_results(models,
                 data,
                 batch_size=128,
                 model_name="vae_mnist"):
    """Plots labels and MNIST digits as function of 2-dim latent vector
    # Arguments:
        models (tuple): encoder and decoder models
        data (tuple): test data and label
        batch_size (int): prediction batch size
        model_name (string): which model is using this function
    """

    encoder, decoder = models
    x_test, y_test = data
    os.makedirs(model_name, exist_ok=True)

    filename = os.path.join(model_name, "vae_mean.png")
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(x_test,
                                   batch_size=batch_size)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y_test)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.savefig(filename)
    plt.show()

    filename = os.path.join(model_name, "digits_over_latent.png")
    # display a 30x30 2D manifold of digits
    n = 30
    digit_size = 256
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = digit

    plt.figure(figsize=(10, 10))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap='Greys_r')
    plt.savefig(filename)
    plt.show()


In [40]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import keras
from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import pydot
import graphviz
import pydotplus

ModuleNotFoundError: ignored

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
!apt-get -qq install -y graphviz && pip install -q pydot
import pydot

Selecting previously unselected package fontconfig.
(Reading database ... 26397 files and directories currently installed.)
Preparing to unpack .../00-fontconfig_2.12.6-0ubuntu2_amd64.deb ...
Unpacking fontconfig (2.12.6-0ubuntu2) ...
Selecting previously unselected package libann0.
Preparing to unpack .../01-libann0_1.1.2+doc-6_amd64.deb ...
Unpacking libann0 (1.1.2+doc-6) ...
Selecting previously unselected package libcdt5.
Preparing to unpack .../02-libcdt5_2.40.1-2_amd64.deb ...
Unpacking libcdt5 (2.40.1-2) ...
Selecting previously unselected package libcgraph6.
Preparing to unpack .../03-libcgraph6_2.40.1-2_amd64.deb ...
Unpacking libcgraph6 (2.40.1-2) ...
Selecting previously unselected package libjbig0:amd64.
Preparing to unpack .../04-libjbig0_2.1-3.1build1_amd64.deb ...
Unpacking libjbig0:amd64 (2.1-3.1build1) ...
Selecting previously unselected package libtiff5:amd64.
Preparing to unpack .../05-libtiff5_4.0.9-5_amd64.deb ...
Unpacking libtiff5:amd64 (4.0.9-5) ...
Selecting pr

In [6]:
 # Then we normalize it so that the values are between 0 and 1
    img_len=256
    img_width=256
    original_dim=img_len*img_width
    x_train = X_train
    x_test = X_test

    
    
    # network parameters
    input_shape = (original_dim, )
    intermediate_dim = 512
    batch_size = 128
    latent_dim = 2
    epochs = 40

    # VAE model = encoder + decoder
    # build encoder model
    inputs = Input(shape=input_shape, name='encoder_input')
    x = Dense(intermediate_dim, activation='relu')(inputs)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)
    
    # use reparameterization trick to push the sampling out as input
    # note that "output_shape" isn't necessary with the TensorFlow backend
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # instantiate encoder model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
    encoder.summary()
    
    plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)

    # build decoder model
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = Dense(intermediate_dim, activation='relu')(latent_inputs)
    outputs = Dense(original_dim, activation='sigmoid')(x)
    
    # instantiate decoder model
    decoder = Model(latent_inputs, outputs, name='decoder')
    decoder.summary()
    plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)
    
    # instantiate VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae_mlp')


    parser = argparse.ArgumentParser()
    help_ = "Load h5 model trained weights"
    parser.add_argument("-w", "--weights", help=help_)
    help_ = "Use mse loss instead of binary cross entropy (default)"
    parser.add_argument("-m",
                        "--mse",
                        help=help_, action='store_true')
    args = parser.parse_args()
    models = (encoder, decoder)
    data = (x_test, y_test)
    print(data)
    

# VAE loss = mse_loss or xent_loss + kl_loss
    if args.mse:
        reconstruction_loss = mse(inputs, outputs)
    else:
        reconstruction_loss = binary_crossentropy(inputs,
                                                  outputs)

    reconstruction_loss *= original_dim
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')
    vae.summary()
    plot_model(vae,
                   to_file='vae_mlp.png',
                   show_shapes=True)
        
    if args.weights:
        vae.load_weights(args.weights)
    else:
        # train the autoencoder
        vae.fit(x_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(x_test, None))
        vae.save_weights('vae_mlp_mnist.h5')
                
        plot_results(models,
                             data,
                             batch_size=batch_size,
                             model_name="vae_mlp")
    


NameError: ignored

***BELOW THIS POINT IS CODE TRIED THAT DOESN'T WORK KEPT FOR REFERENCE***